<h1>Keras Tutorial</h1>
<p>
Theano-based deep learning library for ease of implementing deep neural networks
</p>

In [1]:
from __future__ import absolute_import
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils


<h2>Data Loading</h2>
<p>Load MNIST Hand-written digits data</p>

In [2]:
batch_size = 128
nb_classes = 10
nb_epoch = 20

# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
X_train = X_train.astype("float32")
X_test = X_test.astype("float32")
X_train /= 255
X_test /= 255
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

IOError: [Errno socket error] [Errno 110] Connection timed out

<h2>Model Specification</h2>
<p>Define a simple deep multilayer perceptron. MNIST images are 28x28 pixels (784). The first layer takes 784 inputs to 128 hidden nodes. The hidden layer is then activated using rectified linear unit (ReLU). In addition, a dropout of 20% is added to help avoid overfitting. Next layer is another hidden layer of 128 units, ReLU activation w/ 20% dropout. Finally, the output layer of 10 nodes (10 classes for hand written digits), and a softmax activation function</p>

In [ ]:
model = Sequential()
model.add(Dense(784, 128))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(128, 128))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(128, 10))
model.add(Activation('softmax'))

<h2>Model Compilation</h2>
<p>Compile the model using categorical_crossentropy as a loss function. Optimze using RMSprop, a variant of SGD in which the gradient of the current batch is normalized via average of previous gradients</p>

In [ ]:
rms = RMSprop()
model.compile(loss='categorical_crossentropy', optimizer="RMSprop")

In [5]:
model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch, show_accuracy=True, verbose=2, validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test, show_accuracy=True, verbose=0)

Train on 60000 samples, validate on 10000 samples
Epoch 0
2s - loss: 0.3482 - acc.: 0.8984 - val. loss: 0.2092 - val. acc.: 0.9350
Epoch 1
2s - loss: 0.2007 - acc.: 0.9409 - val. loss: 0.1377 - val. acc.: 0.9574
Epoch 2
2s - loss: 0.1494 - acc.: 0.9551 - val. loss: 0.1077 - val. acc.: 0.9666
Epoch 3
2s - loss: 0.1247 - acc.: 0.9628 - val. loss: 0.0884 - val. acc.: 0.9724
Epoch 4
2s - loss: 0.1075 - acc.: 0.9676 - val. loss: 0.0866 - val. acc.: 0.9713
Epoch 5
2s - loss: 0.0927 - acc.: 0.9727 - val. loss: 0.0760 - val. acc.: 0.9774
Epoch 6
2s - loss: 0.0837 - acc.: 0.9735 - val. loss: 0.0765 - val. acc.: 0.9771
Epoch 7
2s - loss: 0.0754 - acc.: 0.9765 - val. loss: 0.0792 - val. acc.: 0.9754
Epoch 8
2s - loss: 0.0725 - acc.: 0.9777 - val. loss: 0.0735 - val. acc.: 0.9778
Epoch 9
2s - loss: 0.0669 - acc.: 0.9791 - val. loss: 0.0701 - val. acc.: 0.9787
Epoch 10
2s - loss: 0.0619 - acc.: 0.9802 - val. loss: 0.0658 - val. acc.: 0.9798
Epoch 11
2s - loss: 0.0585 - acc.: 0.9813 - val. loss: 0.0

In [6]:
print('Test score:', score[0])
print('Test accuracy:', score[1])

Test score: 0.0591635390648
Test accuracy: 0.984375


<h2>Convolutional Neural Network</h2>
<p>
Train a simple deep NN on the MNIST dataset. Get to 98.30% test accuracy after 20 epochs (there is *a lot* of margin for parameter tuning) .2 seconds per epoch on a GRID K520 GPU.
</p>

In [10]:
# shape of the image (SHAPE x SHAPE)
shapex, shapey = 28, 28
# number of convolutional filters to use
nb_filters = 32
# level of pooling to perform (POOL x POOL)
nb_pool = 2
# level of convolution to perform (CONV x CONV)
nb_conv = 3

In [14]:
# the data, shuffled and split between tran and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], 1, shapex, shapey)
X_test = X_test.reshape(X_test.shape[0], 1, shapex, shapey)
X_train = X_train.astype("float32")
X_test = X_test.astype("float32")
X_train /= 255
X_test /= 255
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

X_train shape: (60000, 1, 28, 28)
60000 train samples
10000 test samples


In [15]:
model = Sequential()

model.add(Convolution2D(nb_filters, 1, nb_conv, nb_conv, border_mode='full'))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters, nb_filters, nb_conv, nb_conv))
model.add(Activation('relu'))
model.add(MaxPooling2D(poolsize=(nb_pool, nb_pool)))
model.add(Dropout(0.25))

model.add(Flatten())
# the resulting image after conv and pooling is the original shape
# divided by the pooling with a number of filters for each "pixel"
# (the number of filters is determined by the last Conv2D)
model.add(Dense(nb_filters * (shapex / nb_pool) * (shapey / nb_pool), 128))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(128, nb_classes))
model.add(Activation('softmax'))

In [16]:
model.compile(loss='categorical_crossentropy', optimizer='adadelta')

In [17]:
model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch, show_accuracy=True, verbose=1, validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test, show_accuracy=True, verbose=0)

Train on 60000 samples, validate on 10000 samples
Epoch 0
60000/60000 [==============================] - 1470s - loss: 2.2029 - acc.: 0.1889 - val. loss: 2.0239 - val. acc.: 0.4230
Epoch 1
60000/60000 [==============================] - 1485s - loss: 2.0922 - acc.: 0.2415 - val. loss: 1.8671 - val. acc.: 0.5081
Epoch 2
60000/60000 [==============================] - 1480s - loss: 2.0090 - acc.: 0.2811 - val. loss: 1.7564 - val. acc.: 0.5755
Epoch 3
60000/60000 [==============================] - 1495s - loss: 1.9465 - acc.: 0.3136 - val. loss: 1.6582 - val. acc.: 0.6144
Epoch 4
60000/60000 [==============================] - 1493s - loss: 1.8876 - acc.: 0.3422 - val. loss: 1.5657 - val. acc.: 0.6490
Epoch 5
60000/60000 [==============================] - 1494s - loss: 1.8492 - acc.: 0.3615 - val. loss: 1.5013 - val. acc.: 0.6665
Epoch 6
60000/60000 [==============================] - 1492s - loss: 1.8150 - acc.: 0.3721 - val. loss: 1.4708 - val. acc.: 0.6807
Epoch 7
60000/60000 [============

In [18]:
print('Test score:', score[0])
print('Test accuracy:', score[1])

Test score: 1.13850177042
Test accuracy: 0.786194620253
